In [ ]:
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
import time

#driver = webdriver.PhantomJS('C:/Users/bnsha_000/Downloads/phantomjs-2.1.1-windows/phantomjs-2.1.1-windows/bin/phantomjs') # or add to your PATH
driver = webdriver.Chrome('C:/Users/bnsha_000/Downloads/chromedriver_win32/chromedriver')
#driver.set_window_size(1024, 768) # optional
url = 'https://www.transtats.bts.gov/DL_SelectFields.asp?Table_ID=236'
driver.get(url)
#driver.save_screenshot('screen.png') # save a screenshot to disk

### Time Period Parameters
Tparam = ['Year','Month','DayofMonth','DayOfWeek','FlightDate']
#### Airline Parameters
Aparam = ['UniqueCarrier','TailNum','FlightNum']
#### Origin Parameters
Oparam = ['OriginAirportID','Origin','OriginCityName','OriginStateName','OriginWac']
#### Destination Parameters
Dparam = ['DestAirportID','Dest','DestCityName','DestStateName','DestWac']
#### Departure Performance
DPfparam = ['CRSDepTime','DepTime','DepDelay','DepartureDelayGroups','TaxiOut','WheelsOff']
#### Arrival Performance
APfparam = ['CRSArrTime','ArrTime','ArrDelay','ArrivalDelayGroups','TaxiIn','WheelsOn']
#### Cancellations
Cparam = ['Cancelled','CancellationCode','Diverted']
#### Flight Summary
Fparam = ['AirTime','Flights','Distance']
### Cause of Delay
CDparam = ['CarrierDelay','WeatherDelay','NASDelay','SecurityDelay','LateAircraftDelay']
### Gate Return Delay
Gparam = ['FirstDepTime','TotalAddGTime','LongestAddGTime']
### Diverted Landings
Dvparam = ['DivAirportLandings']

### Year List
Years = ['2012','2013','2014','2015','2016','2017']
### Months List
Months = ['January','February','March','April','May','June','July','August','September','October','November','December']

param_list = Tparam + Aparam + Oparam + Dparam + DPfparam + APfparam + Cparam + Fparam + CDparam + Gparam + Dvparam

for param in param_list:
    schk = driver.find_element_by_css_selector('input[Title='+param+']')
    webdriver.ActionChains(driver).move_to_element(schk).click(schk).perform()

driver.execute_script("window.scrollTo(0, 0)")

# for year in Years:
#     for month in np.arange(1,13):
#         selectY = Select(driver.find_element_by_id('XYEAR'))
#         selectY.select_by_value(year)
#         selectM = Select(driver.find_element_by_id('FREQUENCY'))
#         selectM.select_by_value(str(month))
#         sbtn = driver.find_element_by_css_selector('button[onclick="tryDownload()"]')
#         sbtn.click()
#         time.sleep(45)

for month in np.arange(1,4):
    selectY = Select(driver.find_element_by_id('XYEAR'))
    selectY.select_by_value('2018')
    selectM = Select(driver.find_element_by_id('FREQUENCY'))
    selectM.select_by_value(str(month))
    sbtn = driver.find_element_by_css_selector('button[onclick="tryDownload()"]')
    sbtn.click()
    time.sleep(45)
driver.close()

In [ ]:
import pandas as pd
import os

path = 'C:/Users/bnsha_000/Desktop/Data Analysis/AirlineDelays/'
data_path = path + '1_Data/zip/'
out_path = path + '1_Data/csvs/'

filelist = os.listdir(data_path)

df_all = pd.DataFrame()
id = 1
for file in filelist:
    print(file)
    temp = pd.read_csv(data_path+file,compression='zip')
    temp.to_csv(out_path+'Fdelay_Data_'+str(id)+'.csv')
    id = id + 1
    #df_all = df_all.append(temp,ignore_index=True)

#df_all.to_csv(path+'DOT_Fdelay_Data.csv')

In [ ]:
import pandas as pd
import os
import pyarrow as pa
import pyarrow.parquet as pq

path = 'C:/Users/bnsha_000/Desktop/Data Analysis/AirlineDelays/'
pq_path = path + '1_Data/parquet/'
csv_path = path + '1_Data/csvs/'

filelist = os.listdir(csv_path)
for file in filelist:
    temp = pd.read_csv(csv_path + file)
    yr = temp['YEAR'].iloc[0]
    mnth = temp['MONTH'].iloc[0]
    fname = 'Fdelay_Data_'+str(yr)+'_'+str(mnth)+'.parquet'
    table = pa.Table.from_pandas(temp)
    pq.write_table(table,fname)
    print(fname)